In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install transformer_lens -q
!pip install --upgrade attrs -q
!pip uninstall jax jaxlib -y -q
!pip install jax jaxlib -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.9.2 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.2.0 which is incompatible.


In [1]:
print("yes")

yes


In [2]:
from transformer_lens import HookedTransformer
import torch

import json
import time

# from src.neuron_heads import head_attribution_over_all_data
from src.datahandlers import ActivatingDataset
from src.utils import tuple_str_to_tuple

In [3]:
import torch
from fancy_einsum import einsum
from tqdm import tqdm

def head_attribution_over_all_data(model, data, device, neurons, batch_size=8):

    head_attribution_dict = {}

    for neuron in tqdm(neurons):
        # Load and Truncate Prompts
        trunc_prompts, prompts_metadata = data.load_truncated_prompts(model, neuron)
        trunc_prompts = trunc_prompts[:20] # 20
        # trunc_prompts = trunc_prompts[:10] # 10
        print(len(trunc_prompts))

        num_batches = len(trunc_prompts) // batch_size + (1 if len(trunc_prompts) % batch_size != 0 else 0)

        head_results = {}
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, len(trunc_prompts))
            current_batch = trunc_prompts[start_idx:end_idx]

            # Run head attribution
            tokens = model.to_tokens(current_batch, prepend_bos=True).to(device=device)
            original_logits, cache = model.run_with_cache(tokens, )

            # Prepare prompts by heads
            head_attribution = _get_head_attribution(model, cache, tokens, neuron) # Gives [batch, head]
            # _, top_heads = torch.topk(head_attribution, k=3, dim=-1)

            sigma = head_attribution.std(dim=-1)
            mean = head_attribution.mean(dim=-1)
            # top_heads_2 = head_attribution[sigma > 2]
            top_heads_2 = torch.where(head_attribution > (mean + 2*sigma).unsqueeze(-1), torch.ones_like(head_attribution), torch.zeros_like(head_attribution))
            top_heads_2_index = torch.nonzero(top_heads_2, as_tuple=True)
            indices, values = top_heads_2_index
            # print(len(current_batch))
            # print(top_heads_2_index)
            top_heads_2_list = []
            if len(indices) == 0:
                top_heads_2_list.append([])
            else:
                for index in indices.unique():
                # Filter and collect values corresponding to the current index
                    top_heads_2_list.append(values[indices == index].tolist())
            # print("Top heads", top_heads_list)
            # print("Top heads 2", top_heads_2_list)

            for i, prompt in enumerate(current_batch):
                head_results[prompt] = top_heads_2_list[i]

        head_attribution_dict[str(neuron)] = head_results

    return head_attribution_dict

def _get_head_attribution(model, cache, tokens, neuron):
    # Get prompt lengths
    if "pythia" in model.cfg.model_name:
        pad_token = 1
        bos_token = 0
    else:
        pad_token = bos_token = 50256 # This is true for GPT-2
    prompt_lengths = (torch.logical_and(tokens != pad_token, tokens != bos_token)).sum(dim=-1)

    # Get the correct last-seq for each prompt (since they are padded, the last seq position differs for each prompt)
    head_output = cache.stack_head_results()
    expanded_index_tensor = prompt_lengths.unsqueeze(0).unsqueeze(-1).unsqueeze(-1).expand(head_output.shape[0], head_output.shape[1], 1, head_output.shape[-1])
    head_output_last_seq = torch.gather(head_output, 2, expanded_index_tensor).squeeze(2)

    # Get dot product of neuron with each head's output
    layer, no = neuron
    neuron_w_in = model.W_in[layer, :, no]
    head_attribution = einsum("head batch weight, weight->batch head", head_output_last_seq, neuron_w_in)
    head_attribution = head_attribution[:, :layer*model.cfg.n_heads] # Filter for only heads before the neuron
    return head_attribution # Shape [batch, head]

In [4]:
# Load jsons from ../experiment_data/1_next_token_neurons
# filename = "2023-10-12_10-51-10_gpt2-large"
# filename = "2023-10-12_06-52-17_gpt2-small"
# filename = "2023-10-13_07-03-35_gpt2-small_test"
# filename = "2023-10-12_07-15-42_gpt2-medium"
# filename = "2023-10-13_07-54-07_gpt2-medium_test"
# filename = "2023-10-13_12-36-52_gpt2-large_test"
# filename = "2023-10-14_08-18-02_gpt2-large_mid"
filename = "2023-10-14_18-20-09_gpt2-large_mid_test"
train = False

# filename = "2024-02-14_01-59-03_pythia-1.4b_train"
# filename = "2024-02-13_17-22-42_pythia-410m_train"
# filename = "2024-02-13_15-37-06_pythia-160m_train"

# filename = "2024-02-15_14-52-20_pythia-1.4b_test_fr"
# filename = "2024-02-15_15-29-26_pythia-410m_test_fr"
# filename = "2024-02-15_15-49-23_pythia-160m_test_fr"

# filename = "2024-02-16_00-34-01_gpt2-large_train_random"
# filename = "2024-02-16_04-53-07_gpt2-large_test_random"


with open(f'./experiment_data/3_trimmed_texts/{filename}.json') as f:
    text_metadata = json.load(f)

neurons = [tuple_str_to_tuple(x) for x in text_metadata['neuron_to_trunc_data'].keys()]

In [ ]:
!ls experiment_data/3_trimmed_texts

/usr/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-10-12_06-52-17_gpt2-small.json
2023-10-12_07-15-42_gpt2-medium.json
2023-10-12_10-51-10_gpt2-large.json
2023-10-12_10-51-25_gpt2-large.json
2023-10-13_07-03-35_gpt2-small_test.json
2023-10-13_07-54-07_gpt2-medium_test.json
2023-10-13_12-36-52_gpt2-large_test.json
2023-10-14_08-18-02_gpt2-large_mid.json
2023-10-14_18-20-09_gpt2-large_mid_test.json
2024-02-13_15-37-06_pythia-160m_train.json
2024-02-13_17-22-42_pythia-410m_train.json
2024-02-14_01-59-03_pythia-1.4b_train.json
2024-02-14_07-59-05_pythia-1.4b_test.json
2024-02-14_08-35-44_pythia-410m_test.json
2024-02-14_08-56-41_pythia-160m_test.json
2024-02-15_14-04-40_pythia-410m_train.json
2024-02-15_14-52-20_pythia-1.4b_test_fr.json
2024-02-15_15-29-26_pythia-410m_test_fr.json
2024-02-15_15-49-23_pythia-160m_test_fr.json


In [5]:
if train:

    from datasets import load_dataset
    dataset = load_dataset("NeelNanda/pile-10k", split="train")
    dataset_text_list = [x['text'] for x in dataset]
    print("Train Loaded!")

else:
    import pickle

    with open('./text_list_dict_test.pkl', 'rb') as f:
        dataset_text_dict = pickle.load(f)

    dataset_text_list = [x['text'] for x in dataset_text_dict]
    dataset = dataset_text_dict
    print("Test Loaded!")

data = ActivatingDataset(text_metadata['neuron_to_trunc_data'], dataset)
data.remove_prompts_longer_than(100)

Test Loaded!


In [6]:
# Parameters
parameters = text_metadata['parameters']
model_name = parameters['model_name']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [7]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    # refactor_factored_attn_matrices=True,
    device=device,
)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-large into HookedTransformer


In [8]:
with torch.no_grad():
    head_attribution_dict = head_attribution_over_all_data(
        model=model,
        data=data,
        device=device,
        neurons=neurons,
        batch_size=1,
    )

  0%|          | 0/100 [00:00<?, ?it/s]

20
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head resul

  1%|          | 1/100 [00:03<05:43,  3.47s/it]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
20
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head resul

  2%|▏         | 2/100 [00:07<06:01,  3.69s/it]

...

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
20
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head resul

 99%|█████████▉| 99/100 [05:26<00:03,  3.28s/it]

Tried to stack head results when they weren't cached. Computing head results now
20
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head resul

100%|██████████| 100/100 [05:29<00:00,  3.30s/it]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


In [9]:
output = {
    'parameters': parameters,
    'head_attributions': head_attribution_dict,
    'prior_filename': filename,
}

train_str = "train" if train else "test"

# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{timestamp}_{model_name}_{train_str}_aclcn.json"

with open(f'./experiment_data/4_head_attributions/{new_filename}', 'w') as f:
    json.dump(output, f)

In [ ]:
import json

with open("../experiment_data/4_head_attributions/2023-10-12_14-43-12_gpt2-small.json") as f:
    output1 = json.load(f)

with open("../experiment_data/4_head_attributions/2023-10-12_22-00-53_gpt2-small.json") as f:
    output2 = json.load(f)

# Check if two dictionaries are the same throughout
output1 == output2